# Downloading data

In [ ]:
!kaggle datasets download -d sartazansari/e-commerce-sales-dataset

# Unzipping dataset

In [ ]:
import zipfile
with zipfile.ZipFile("e-commerce-sales-dataset.zip", "r") as file:
    file.extractall("Raw Dataset")

# Importing packages

In [ ]:
import os
import pprint
import pandas as pd
import numpy as np

# Defining paths

In [ ]:
project_dir = "D:\\Projects\\Python\\ecom-data-clean"
raw_dir = os.path.join(project_dir, "Raw Dataset")
cleaned_dir = os.path.join(project_dir, "Clean Dataset")
data_folders = ["Expense", "Inventory", "Pricing", "Sales", "Warehouse"]

# Getting files in dictionary

In [ ]:
def get_data_from_folders(folders):
    
    folder_files = {}
    
    for i, folder in enumerate(folders):
        folder = os.path.join(raw_dir, folder)
        
        for root, dirs, files in os.walk(folder):
            files_arr = []
            
            for file in files:
                files_arr.append(os.path.join(root, file))
                
            folder_files[data_folders[i]] = files_arr
    return folder_files
                

data_files = get_data_from_folders(data_folders)

pprint.pprint(data_files)

# Reading files

In [ ]:
def read_data(files):
    for key, value in files.items():
        print(f"{key}\n")
        for item in value:
            df = pd.read_csv(item, low_memory=False)
            display(df.info())
            display(df.head(5))
            display(df.tail(5))
            display(df.sample(5))
            
read_data(data_files)  

# Cleaning files

In [ ]:
def clean_warehouse(df):
    df = df.drop(columns=["index"])
    df = df.drop(columns=df.columns[2])
    df.columns = df.iloc[0]
    df = df.drop([0])
    nan_index = df.iloc[:, 1].isna().idxmax()
    df = df.iloc[:nan_index-1]
    df.iloc[:, 1] = df.iloc[:, 1].str.replace("₹", "")
    df.iloc[:, 1] = df.iloc[:, 1].apply(pd.to_numeric, errors="coerce")
    return df

def clean_sales(df):
    col = df.columns[df.columns.str.contains("Unnamed")]
    df = df.drop(columns=["index", *col])
    col = df.columns[df.columns.str.contains("Date")]
    df[col] = df[col].apply(pd.to_datetime, format="%m-%d-%y", errors="coerce")
    col = df.columns[df.columns.str.contains("Qty")]
    df[col] = df[col].apply(pd.to_numeric, errors="coerce")
    return df

def clean_sales_international(df):
    df = df.drop(columns=["index"])
    header = ["CUSTOMER", "DATE", "Months", "Style", "SKU", "PCS", "RATE", "GROSS AMT", "Stock"]
    temp_list = []
    start = 0
    row_with_header = True
    for i in range(len(df)):
        row = df.iloc[i].dropna().tolist()
        if row and set(row).issubset(set(header)):
            if np.array_equal(np.sort(header), np.sort(row)):
                start = i+1
                row_with_header = True
                continue
            elif row_with_header:
                    temp_df = df[start : i]
                    temp_df.columns = header
                    row_with_header = False
        elif i == len(df)-1:
            temp_df = df[start:]
            temp_df.columns = header
            temp_list.append(temp_df)
    df = pd.concat(temp_list)
    col = df.columns[df.columns.str.contains("Date")]
    df[col] = df[col].apply(pd.to_datetime, format="%m-%d-%y", errors="coerce")
    col = df.columns[df.columns.str.contains("Months")]
    df[col] = df[col].apply(pd.to_datetime, format="%b-%y", errors="coerce")
    col = df.columns[df.columns.str.contains("|".join(["PCS", "RATE", "GROSS AMT"]))]
    df[col] = df[col].apply(pd.to_numeric, errors="coerce")
    return df

def clean_pricing(df):
    df = df.drop(columns="index")
    col = df.columns[df.columns.str.contains("|".join(["MRP", "TP", "Weight"]))]
    df[col] = df[col].apply(pd.to_numeric, errors="coerce")
    df[col] = df[col].fillna(0)
    return df

def clean_inventory(df):
    df = df.drop(columns="index")
    col = df.columns[df.columns.str.contains("Stock")]
    df[col] = df[col].apply(pd.to_numeric, errors="coerce")
    return df

def clean_expense(df):
    df = df.drop(columns="index")
    df = df.drop(columns=df.columns[0:1])
    col = df.columns[1]
    df[col] = df[col].apply(pd.to_numeric, errors="coerce")
    df = df.dropna(subset=df.columns[1])
    df.rename(columns=lambda col: "Expense" if "Expance" in col else col, inplace=True)
    df.rename(columns=lambda col: "Amount" if "Unnamed" in col else col, inplace=True)
    return df

def clean_data(files):
    for key, value in files.items():
        print(key)
        for item in value:
            df = pd.read_csv(item, header=0, low_memory=False)
            df = df.dropna(how="all")
            df = df.drop_duplicates()
            match key:
                case "Expense":
                    df = clean_expense(df)
                case "Inventory":
                    df = clean_inventory(df)
                case "Pricing":
                    df = clean_pricing(df)
                case "Sales":
                    if "International" in item:
                        df = clean_sales_international(df)
                    else:
                        df = clean_sales(df)
                case "Warehouse":
                    df = clean_warehouse(df)
            filename = "Cleaned_" + os.path.basename(item)
            directory = os.path.join(cleaned_dir, key)
            if not os.path.exists(directory):
                os.makedirs(directory)
            df.to_csv(os.path.join(directory, filename), index=False)
            display(df) 
            

clean_data(data_files)            